In [1]:
# SCRIPT 0: IMPORT LIBRARIES
import yfinance as yf
import pandas as pd
import numpy as np
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS
from stargazer.stargazer import Stargazer
import os

In [2]:
# SCRIPT 1: SETUP - DEFINE CONSTANTS AND TICKERS

TICKERS = {
    "Large Cap": [
        'EQNR.OL', 'DNB.OL', 'KOG.OL', 'MOWI.OL', 'TEL.OL', 'NHY.OL', 'AKRBP.OL',
        'ORK.OL', 'STB.OL', 'YAR.OL', 'SUBC.OL', 'GJF.OL', 'SALM.OL', 'TGS.OL',
        'TOM.OL', 'VAR.OL', 'NEL.OL', 'FRO.OL', 'BWLPG.OL', 'HAUTO.OL',
        'NOD.OL', 'WAWI.OL', 'NAS.OL', 'BAKKA.OL', 'WWI.OL', 'AFK.OL',
        'AUSS.OL', 'SCATC.OL', 'MPCC.OL', 'HAFNI.OL'
    ],
    "Mid Cap": [
        'AKER.OL', 'LSG.OL', 'KIT.OL', 'AKSO.OL', 'PARB.OL', 'BONHR.OL',
        'BOUV.OL', 'DNO.OL', 'ENTRA.OL', 'FLNG.OL', 'MING.OL', 'NAPA.OL',
        'NORBT.OL', 'OLT.OL', 'PCIB.OL', 'REACH.OL', 'WSTEP.OL', 'KOA.OL',
        'HSPG.OL', 'SOFF.OL', 'ABG.OL', 'BGBIO.OL', 'EMGS.OL', 'EXTX.OL',
        'HAVI.OL', 'HELG.OL', 'IDEX.OL', 'JIN.OL', 'MULTI.OL', 'NYKD.OL'
    ],
    "Small Cap": [
        'QEC.OL', 'RECSI.OL', 'SPOL.OL', 'AZT.OL', 'KID.OL', 'SATS.OL',
        'AURG.OL', 'PEN.OL', 'LINK.OL', 'PROT.OL', 'IOX.OL', 'ACC.OL',
        'TECH.OL', 'CONTX.OL', 'NONG.OL', 'BEWI.OL', 'ELO.OL', 'GSF.OL',
        'PRS.OL', 'AIRX.OL', 'OBSRV.OL', 'HUNT.OL', 'AKVA.OL', 'HEX.OL',
        'SOFTX.OL', 'ASA.OL', 'NORTH.OL', 'CAPSL.OL', 'LYTIX.OL', 'VOW.OL'
    ]
}

START_DATE = "2014-01-01"
END_DATE = "2024-12-31"
OUTPUT_CSV = "oslo_bors_labelled_data.csv"


In [ ]:
# SCRIPT 1.1: CORWIN-SCHULTZ, ABDI-RANALDO, AND ROLL SPREAD ESTIMATORS
def corwin_schultz_spread(group):
    """
    Corwin-Schultz (2012) High-Low spread estimator.
    This function calculates a daily spread estimate.
    
    Reference: Corwin, S. A., & Schultz, P. (2012). A simple way to estimate 
    bid-ask spreads from daily high and low prices. The Journal of Finance, 67(2), 719-760.
    """
    group = group.sort_values('Date')
    
    high = group['High'].values
    low = group['Low'].values
    
    spreads = []
    negative_count = 0
    
    for i in range(len(group)):
        if i == 0:
            spreads.append(np.nan)
        else:
            h0, l0 = high[i], low[i]
            h1, l1 = high[i-1], low[i-1]
            
            if h0 > 0 and l0 > 0 and h1 > 0 and l1 > 0:
                beta = (np.log(h0/l0))**2 + (np.log(h1/l1))**2
                h_max = max(h0, h1)
                l_min = min(l0, l1)
                gamma = (np.log(h_max/l_min))**2
                
                k = (3 - 2*np.sqrt(2))
                alpha = (np.sqrt(2*beta) - np.sqrt(beta)) / k - np.sqrt(gamma / k)
                
                spread = 2 * (np.exp(alpha) - 1) / (1 + np.exp(alpha))
                
                if spread >= 0:
                    spreads.append(spread)
                else:
                    spreads.append(np.nan)
                    negative_count += 1
            else:
                spreads.append(np.nan)
    
    if negative_count > 0:
        ticker = group['Ticker'].iloc[0] if 'Ticker' in group.columns else 'Unknown'
        # print(f"  Warning: {ticker} had {negative_count} negative Corwin-Schultz estimates (set to NaN)")
    
    group['Spread'] = spreads
    return group



def abdi_ranaldo_spread(group):
    """
    Calculates the NORMALIZED Abdi and Ranaldo (2017) "BAR" spread estimator.
    This version converts the absolute spread (in currency) to a relative 
    spread (as a percentage of the price), making it comparable across stocks.
    
    Note: Additional filtering is applied to remove economically meaningless spreads.
    """
    group = group.sort_values('Date')

    delta_p_t = group['Close'].diff()
    delta_p_t_minus_1 = delta_p_t.shift(1)

    if delta_p_t.count() < 2:
        return pd.Series(np.nan, index=group.index, name='Spread')

    cov_term = (delta_p_t * delta_p_t_minus_1)
    
    # Calculate the absolute spread (in currency units)
    absolute_spreads = 2 * np.sqrt(np.maximum(0, -cov_term))
    
    # NORMALIZATION STEP
    # To get a relative spread, divide by the average of today's and yesterday's close price.
    # Rolling window to get the average price at each point in time.
    avg_price = group['Close'].rolling(window=2, min_periods=1).mean()
    
    # Calculate the relative spread. Use .values to avoid index alignment issues.
    relative_spreads = absolute_spreads / avg_price.values
    
    # CRITICAL FIX: Filter out economically meaningless spreads
    # Spreads below 0.01% (0.0001) are likely measurement errors or rounding artifacts
    relative_spreads = np.where(relative_spreads < 0.0001, np.nan, relative_spreads)
    
    # Assign the comparable, relative spreads to the DataFrame
    group['Spread'] = relative_spreads
    return group



def get_roll_spread(group):
    """
    Roll (1984) spread estimator based on serial covariance of price changes.
    Returns a single spread estimate per stock (not time-varying).
    
    Roll's spread = 2 * sqrt(-Cov(ΔP_t, ΔP_t-1))
    
    The Roll estimator assumes that negative serial covariance in returns is 
    caused by bid-ask bounce. If the covariance is positive, it violates 
    Roll's model assumptions and we return NaN.
    
    Reference: Roll, R. (1984). A simple implicit measure of the effective 
    bid-ask spread in an efficient market. The Journal of Finance, 39(4), 1127-1139.
    """
    group = group.sort_values('Date')
    returns = group['Close'].pct_change().dropna()
    
    if len(returns) < 30:  # Minimum observations for reliable estimate
        return np.nan
    
    # Calculate serial covariance (covariance between returns and lagged returns)
    cov = returns.cov(returns.shift(1))
    
    # Roll's spread formula (only valid when covariance is negative)
    if pd.notna(cov) and cov < 0:
        spread = 2 * np.sqrt(-cov)
    else:
        # Positive covariance violates Roll's assumptions
        spread = np.nan
    
    return spread


In [4]:
# SCRIPT 1.2: ADD PERIOD LABELS FOR HOLIDAY ANALYSIS
def add_period_labels(df):
    """
    Adds period labels for holiday analysis.
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    years = range(df['Date'].dt.year.min(), df['Date'].dt.year.max() + 1)
    norway_holidays = holidays.Norway(years=years)
    
    df['week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['month'] = df['Date'].dt.month
    df['period_label'] = 'control_year'

    for yr in years:
        christmas_ref_start = pd.Timestamp(f'{yr}-12-15')
        christmas_ref_end = pd.Timestamp(f'{yr}-12-31')
        christmas_days = pd.bdate_range(start=christmas_ref_start, end=christmas_ref_end, freq='C', holidays=list(norway_holidays.keys()), weekmask='Mon Tue Wed Thu Fri')
        
        if len(christmas_days) > 0:
            pre_christmas = pd.bdate_range(end=christmas_days[0] - pd.Timedelta(days=1), periods=5, freq='C', holidays=list(norway_holidays.keys()), weekmask='Mon Tue Wed Thu Fri')
            try:
                post_christmas_start = pd.Timestamp(f'{yr+1}-01-02')
                post_christmas = pd.bdate_range(start=post_christmas_start, periods=5, freq='C', holidays=list(norway_holidays.keys()), weekmask='Mon Tue Wed Thu Fri')
            except:
                post_christmas = pd.bdate_range(start=christmas_days[-1] + pd.Timedelta(days=1), periods=5, freq='C', holidays=list(norway_holidays.keys()), weekmask='Mon Tue Wed Thu Fri')
            
            df.loc[df['Date'].isin(pre_christmas), 'period_label'] = 'pre_christmas'
            df.loc[df['Date'].isin(christmas_days), 'period_label'] = 'christmas'
            df.loc[df['Date'].isin(post_christmas), 'period_label'] = 'post_christmas'

    for yr in years:
        easter_days = [d for d, h in norway_holidays.items() if ('Påske' in h or h in ['Skjærtorsdag', 'Langfredag']) and d.year == yr]
        
        if easter_days:
            easter_start = min(easter_days)
            easter_end = max(easter_days)
            pre_easter = pd.bdate_range(end=easter_start - pd.Timedelta(days=1), periods=5, freq='C', holidays=list(norway_holidays.keys()), weekmask='Mon Tue Wed Thu Fri')
            post_easter = pd.bdate_range(start=easter_end + pd.Timedelta(days=1), periods=5, freq='C', holidays=list(norway_holidays.keys()), weekmask='Mon Tue Wed Thu Fri')
            df.loc[df['Date'].isin(pre_easter), 'period_label'] = 'pre_easter'
            df.loc[df['Date'].isin(post_easter), 'period_label'] = 'post_easter'

    df.loc[df['week'].between(28, 30), 'period_label'] = 'summer_holiday'
    df.loc[(df['month'].between(6, 8)) & (~df['week'].between(28, 30)), 'period_label'] = 'summer_excl_holiday'

    return df


In [5]:
# SCRIPT 1.3: CHOOSE SPREAD ESTIMATOR (Simplified)
user_input = input("Which spread estimator do you want to use? (Enter 'CS' or 'AR'): ")

# Determine the spread method based on user input
if user_input.upper() == 'AR':
    METHOD_CHOICE = 'AR'
    SPREAD_METHOD_NAME = 'Abdi & Ranaldo (2017) Spread'
else:
    METHOD_CHOICE = 'CS'
    SPREAD_METHOD_NAME = 'Corwin-Schultz (2012) Spread'

print(f"The analysis will use the '{SPREAD_METHOD_NAME}'")


The analysis will use the 'Corwin-Schultz (2012) Spread'


In [6]:
# SCRIPT 1.4: MAIN DATA PROCESSING FUNCTION
def main():
    print("-" * 80)
    print("OSLO BØRS HOLIDAY EFFECTS ANALYSIS - DATA PREPARATION")
    # Use f-string to print the full name
    print(f"Using spread estimator: {SPREAD_METHOD_NAME}")
    print("-" * 80)
    
    all_tickers = [ticker for sublist in TICKERS.values() for ticker in sublist]
    print(f"\nDownloading data for {len(all_tickers)} tickers...")
    print(f"Date range: {START_DATE} to {END_DATE}")
    
    raw_data = yf.download(all_tickers, start=START_DATE, end=END_DATE, progress=True, auto_adjust=True)

    if raw_data.empty:
        print("ERROR: No data downloaded")
        return

    print("\n" + "-"*80)
    print("TRANSFORMING DATA STRUCTURE")
    print("-"*80)
    df = raw_data.stack(future_stack=True).reset_index()
    df = df.rename(columns={'level_1': 'Ticker'})
    
    ticker_to_cap = {ticker: cap for cap, tickers in TICKERS.items() for ticker in tickers}
    df['Cap_Group'] = df['Ticker'].map(ticker_to_cap)
    
    initial_len = len(df)
    df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume', 'Cap_Group'], inplace=True)
    print(f"Removed {initial_len - len(df):,} rows with missing price/volume data")

    # Spread Calculation
    print("\n" + "-"*80)
    if METHOD_CHOICE == 'CS':
        print("CALCULATING CORWIN-SCHULTZ SPREADS (DAILY ESTIMATE)")
        df = df.groupby('Ticker', group_keys=False).apply(corwin_schultz_spread)
    elif METHOD_CHOICE == 'AR':
        print("CALCULATING ABDI & RANALDO (2017) 'BAR' SPREADS")
        df = df.groupby('Ticker', group_keys=False).apply(abdi_ranaldo_spread)
    else:
        raise ValueError("Invalid method choice. Please choose 'CS' or 'AR'.")
    print("Spread calculation complete.")

    # CRITICAL FIX: COMPREHENSIVE SPREAD DIAGNOSTICS
    print("\n" + "="*80)
    print(f"📊 SPREAD DISTRIBUTION DIAGNOSTICS - {METHOD_CHOICE}")
    print("="*80)
    
    # Count spread values by category
    total_obs = len(df)
    zero_spreads = (df['Spread'] == 0).sum()
    negative_spreads = (df['Spread'] < 0).sum()
    tiny_spreads = ((df['Spread'] > 0) & (df['Spread'] < 0.0001)).sum()
    valid_spreads = (df['Spread'] >= 0.0001).sum()
    nan_spreads = df['Spread'].isna().sum()
    
    print(f"\nSpread Value Distribution:")
    print(f"  Total observations:        {total_obs:,}")
    print(f"  Zero values (= 0):         {zero_spreads:,} ({zero_spreads/total_obs*100:.2f}%)")
    print(f"  Negative values (< 0):     {negative_spreads:,} ({negative_spreads/total_obs*100:.2f}%)")
    print(f"  Tiny spreads (0 to 0.01%): {tiny_spreads:,} ({tiny_spreads/total_obs*100:.2f}%)")
    print(f"  Valid spreads (≥ 0.01%):   {valid_spreads:,} ({valid_spreads/total_obs*100:.2f}%)")
    print(f"  Missing (NaN):             {nan_spreads:,} ({nan_spreads/total_obs*100:.2f}%)")
    
    # Percentile analysis
    print(f"\nSpread Percentiles (before filtering):")
    percentiles = df['Spread'].describe(percentiles=[.01, .05, .10, .25, .50, .75, .90, .95, .99])
    for stat in ['min', '1%', '5%', '10%', '25%', '50%', '75%', '90%', '95%', '99%', 'max']:
        if stat in percentiles.index:
            print(f"  {stat:>5}: {percentiles[stat]:.6f}")
    
    # Warning if median is zero or very low
    median_spread = df['Spread'].median()
    if median_spread == 0:
        print("\n⚠️  WARNING: Median spread is ZERO!")
        print("   → This indicates a data quality issue with the spread estimator.")
        print("   → Most observations have zero spread, which is economically unrealistic.")
    elif median_spread < 0.0001:
        print(f"\n⚠️  WARNING: Median spread is very low ({median_spread:.6f})")
        print("   → Many spreads may be economically meaningless.")
    else:
        print(f"\n✓ Median spread: {median_spread:.6f} (appears reasonable)")

    # Data Quality Check and Cleaning
    print("\n" + "-"*80)
    print(f"DATA QUALITY FILTERING - {METHOD_CHOICE}")
    print("-" * 80)
    
    spread_before = len(df)
    
    # Remove NaN spreads
    df = df.dropna(subset=['Spread'])
    nan_removed = spread_before - len(df)
    print(f"\nStep 1: Removed {nan_removed:,} rows with NaN spreads")
    
    # CRITICAL FIX: Remove economically meaningless spreads
    # Spreads below 0.01% (1 basis point) are likely measurement errors
    spread_before_filter = len(df)
    df = df[df['Spread'] >= 0.0001]
    tiny_removed = spread_before_filter - len(df)
    print(f"Step 2: Removed {tiny_removed:,} rows with spreads < 0.01% (1 bp)")
    print(f"        → These are economically meaningless and likely measurement errors")
    
    print(f"\nTotal rows removed: {spread_before - len(df):,}")
    print(f"Remaining observations: {len(df):,}")
    
    # Post-filtering diagnostics
    print(f"\n📊 POST-FILTERING SPREAD STATISTICS:")
    print(f"  Mean:   {df['Spread'].mean():.6f}")
    print(f"  Median: {df['Spread'].median():.6f}")
    print(f"  Std:    {df['Spread'].std():.6f}")
    print(f"  Min:    {df['Spread'].min():.6f}")
    print(f"  Max:    {df['Spread'].max():.6f}")

    # Roll's Spread Calculation
    print("\n" + "-" * 80)
    print("CALCULATING ROLL'S SPREADS (VALIDATION)")
    print("-" * 80)
    roll_spreads = df.groupby('Ticker').apply(get_roll_spread)
    df['RollsSpread'] = df['Ticker'].map(roll_spreads)
    
    valid_rolls = roll_spreads.notna().sum()
    total_tickers = df['Ticker'].nunique()
    invalid_rolls = total_tickers - valid_rolls
    
    print(f"\nRoll's spread calculation summary:")
    print(f"  Tickers with valid spread: {valid_rolls} / {total_tickers}")
    if invalid_rolls > 0:
        print(f"  Tickers with invalid spread: {invalid_rolls}")
        print(f"    → Likely due to positive serial covariance (violates Roll's assumptions)")
    print(f"  Average Roll's spread: {roll_spreads.mean():.6f}")
    
    # Compare Roll with CS/AR
    avg_cs_ar = df.groupby('Ticker')['Spread'].mean()
    comparison = pd.DataFrame({
        'Roll': roll_spreads,
        'CS_AR_Avg': avg_cs_ar
    }).dropna()
    
    if len(comparison) > 0:
        correlation = comparison.corr().iloc[0, 1]
        print(f"\n✓ Correlation between Roll and {METHOD_CHOICE} average: {correlation:.3f}")
        if correlation > 0.7:
            print(f"  → Strong correlation indicates reliable spread estimates!")
        elif correlation > 0.5:
            print(f"  → Moderate correlation - spread estimates are reasonably consistent")
        else:
            print(f"  ⚠️  Low correlation - may indicate measurement issues")

    # Period Labeling and Final Export
    print("\n" + "-"*80)
    print("ADDING PERIOD LABELS")
    print("-"*80)
    df = add_period_labels(df)
    
    print("\n" + "-"*80)
    print("FINAL DATA EXPORT")
    print("-"*80)
    
    final_cols = ['Date', 'Ticker', 'Cap_Group', 'Open', 'High', 'Low', 'Close', 
                  'Volume', 'Spread', 'RollsSpread', 'period_label']
    df_final = df[[col for col in final_cols if col in df.columns]]
    df_final.to_csv(OUTPUT_CSV, index=False)
    
    print(f"\nData saved to: {OUTPUT_CSV}")
    print(f"Total observations: {len(df_final):,}")
    print(f"Date range: {df_final['Date'].min()} to {df_final['Date'].max()}")
    print(f"Tickers: {df_final['Ticker'].nunique()}")
    
    # Final summary by cap group
    print(f"\n📊 FINAL SPREAD SUMMARY BY CAP GROUP:")
    summary = df_final.groupby('Cap_Group')['Spread'].agg(['count', 'mean', 'median', 'std'])
    summary.columns = ['Observations', 'Mean', 'Median', 'Std Dev']
    print(summary.to_string())
    
    print("\n" + "-"*80)
    print("✓ DATA PROCESSING COMPLETE")
    print("-"*80)


if __name__ == "__main__":
    main()


--------------------------------------------------------------------------------
OSLO BØRS HOLIDAY EFFECTS ANALYSIS - DATA PREPARATION
Using spread estimator: Corwin-Schultz (2012) Spread
--------------------------------------------------------------------------------

Date range: 2014-01-01 to 2024-12-31


[*********************100%***********************]  90 of 90 completed



--------------------------------------------------------------------------------
TRANSFORMING DATA STRUCTURE
--------------------------------------------------------------------------------
Removed 33,216 rows with missing price/volume data

--------------------------------------------------------------------------------
CALCULATING CORWIN-SCHULTZ SPREADS (DAILY ESTIMATE)


/var/folders/4f/whd5k2z14l1f9v052bj7zhfm0000gn/T/ipykernel_16338/968052951.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Ticker', group_keys=False).apply(corwin_schultz_spread)
/var/folders/4f/whd5k2z14l1f9v052bj7zhfm0000gn/T/ipykernel_16338/968052951.py:119: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  roll_spreads = df.groupby('Ticker').apply(get_roll_spread)


Spread calculation complete.

📊 SPREAD DISTRIBUTION DIAGNOSTICS - CS

Spread Value Distribution:
  Total observations:        215,364
  Zero values (= 0):         11,986 (5.57%)
  Negative values (< 0):     0 (0.00%)
  Tiny spreads (0 to 0.01%): 1,215 (0.56%)
  Valid spreads (≥ 0.01%):   119,138 (55.32%)
  Missing (NaN):             83,025 (38.55%)

Spread Percentiles (before filtering):
    min: 0.000000
     1%: 0.000000
     5%: 0.000000
    10%: 0.000108
    25%: 0.004321
    50%: 0.010319
    75%: 0.019523
    90%: 0.033346
    95%: 0.045516
    99%: 0.080000
    max: 0.408711

✓ Median spread: 0.010319 (appears reasonable)

--------------------------------------------------------------------------------
DATA QUALITY FILTERING - CS
--------------------------------------------------------------------------------

Step 1: Removed 83,025 rows with NaN spreads
Step 2: Removed 13,201 rows with spreads < 0.01% (1 bp)
        → These are economically meaningless and likely measurement er

In [7]:
# SCRIPT 2: DESCRIPTIVE STATISTICS AND VISUALIZATION (ENHANCED)

df = pd.read_csv('oslo_bors_labelled_data.csv', parse_dates=['Date'])

if not os.path.exists('output'):
    os.makedirs('output')

sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.2)

print("-"*80)
print("DESCRIPTIVE STATISTICS AND VISUALIZATION")
print("-"*80)

# Dataset overview
print(f"\nTotal observations: {len(df):,}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Unique tickers: {df['Ticker'].nunique()}")
print(f"Trading days: {df['Date'].nunique():,}")

# CRITICAL ADDITION: COHEN'S D EFFECT SIZE CALCULATION
print("\n" + "="*80)
print("EFFECT SIZE ANALYSIS (COHEN'S D)")
print("="*80)

def calculate_cohens_d(group1, group2):
    """
    Calculate Cohen's d effect size.
    
    Interpretation:
    - d < 0.2: Small effect
    - d = 0.2-0.5: Small to medium effect
    - d = 0.5-0.8: Medium to large effect
    - d > 0.8: Large effect
    """
    n1, n2 = len(group1), len(group2)
    var1, var2 = group1.var(), group2.var()
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    
    # Cohen's d
    d = (group1.mean() - group2.mean()) / pooled_std
    
    return d

# Calculate effect sizes for key holiday periods
control = df[df['period_label'] == 'control_year']['Spread']
summer = df[df['period_label'] == 'summer_holiday']['Spread']
christmas = df[df['period_label'] == 'christmas']['Spread']
easter_pre = df[df['period_label'] == 'pre_easter']['Spread']
easter_post = df[df['period_label'] == 'post_easter']['Spread']

print("\nCohen's d Effect Sizes (vs. Control Period):")
print(f"  Summer Holiday:    d = {calculate_cohens_d(summer, control):.4f}")
print(f"  Christmas:         d = {calculate_cohens_d(christmas, control):.4f}")
print(f"  Pre-Easter:        d = {calculate_cohens_d(easter_pre, control):.4f}")
print(f"  Post-Easter:       d = {calculate_cohens_d(easter_post, control):.4f}")
print("\nInterpretation: |d| > 0.5 = medium to large effect")

# CRITICAL ADDITION: HARRIS FRAMEWORK - DEALER BEHAVIOR HYPOTHESIS TEST
print("\n" + "="*80)
print("HARRIS FRAMEWORK: DEALER BEHAVIOR HYPOTHESIS")
print("="*80)
print("\nTesting Harris's prediction: Spreads should increase when volume decreases")
print("(Lower dealer participation and higher inventory risk)")

# Create volume quartiles
df['Volume_Quartile'] = pd.qcut(df['Volume'], q=4, labels=['Q1_Lowest', 'Q2_Low', 'Q3_High', 'Q4_Highest'], duplicates='drop')

# Spread by volume quartile
spread_by_volume = df.groupby('Volume_Quartile', observed=True).agg({
    'Spread': ['mean', 'median', 'count'],
    'Volume': 'mean'
}).round(6)
spread_by_volume.columns = ['Spread_Mean', 'Spread_Median', 'N_Obs', 'Avg_Volume']

print("\nSpread by Volume Quartile:")
print(spread_by_volume)

# Statistical test: Is spread in Q1 (lowest volume) significantly higher than Q4?
q1_spread = df[df['Volume_Quartile'] == 'Q1_Lowest']['Spread']
q4_spread = df[df['Volume_Quartile'] == 'Q4_Highest']['Spread']
t_stat, p_val = ttest_ind(q1_spread.dropna(), q4_spread.dropna(), equal_var=False)
cohens_d_volume = calculate_cohens_d(q1_spread.dropna(), q4_spread.dropna())

print(f"\nHarris Hypothesis Test:")
print(f"  H0: Spread(Low Volume) = Spread(High Volume)")
print(f"  H1: Spread(Low Volume) > Spread(High Volume)")
print(f"  Mean spread Q1 (low vol):  {q1_spread.mean():.6f}")
print(f"  Mean spread Q4 (high vol): {q4_spread.mean():.6f}")
print(f"  Difference: {q1_spread.mean() - q4_spread.mean():.6f}")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_val:.4f}")
print(f"  Cohen's d: {cohens_d_volume:.4f}")

if p_val < 0.05 and q1_spread.mean() > q4_spread.mean():
    print("  ✓ RESULT: Harris's prediction CONFIRMED (p < 0.05)")
    print("    → Spreads are wider when volume is low (dealer participation reduced)")
elif p_val < 0.05 and q1_spread.mean() < q4_spread.mean():
    print("  ✗ RESULT: Harris's prediction CONTRADICTED (p < 0.05)")
    print("    → Modern electronic markets may behave differently than Harris predicted")
else:
    print("  → RESULT: No significant relationship (p >= 0.05)")

# Summary by cap group
print("\n" + "-"*80)
print("SUMMARY BY CAP GROUP")
print("-"*80)
summary_cap = df.groupby('Cap_Group').agg({
    'Spread': ['mean', 'median', 'std'],
    'Volume': ['mean', 'median'],
    'Ticker': 'nunique'
}).round(6)
summary_cap.columns = ['Spread_Mean', 'Spread_Median', 'Spread_Std', 'Volume_Mean', 'Volume_Median', 'N_Stocks']
print(summary_cap)

# HARRIS INTERPRETATION
print("\n📖 HARRIS INTERPRETATION:")
print("   Large Cap stocks have narrower spreads due to:")
print("   • Higher competitive market making (Harris Chapter 6)")
print("   • Lower inventory risk for dealers")
print("   • Greater participation rates (Harris Chapter 21)")

# Summary by period
print("\n" + "-"*80)
print("SUMMARY BY PERIOD")
print("-"*80)
summary_period = df.groupby('period_label').agg({
    'Spread': ['mean', 'median'],
    'Volume': ['mean', 'median'],
    'Date': 'count'
}).round(6)
summary_period.columns = ['Spread_Mean', 'Spread_Median', 'Volume_Mean', 'Volume_Median', 'N_Obs']
print(summary_period)

# Calculate percentage changes vs control
control_spread = summary_period.loc['control_year', 'Spread_Mean']
control_volume = summary_period.loc['control_year', 'Volume_Mean']
summary_period['Spread_Change_%'] = ((summary_period['Spread_Mean'] - control_spread) / control_spread * 100).round(2)
summary_period['Volume_Change_%'] = ((summary_period['Volume_Mean'] - control_volume) / control_volume * 100).round(2)

print("\n" + "-"*80)
print("CHANGES VS CONTROL PERIOD")
print("-"*80)
print(summary_period[['Spread_Mean', 'Spread_Change_%', 'Volume_Mean', 'Volume_Change_%']])

summary_cap.to_csv('output/summary_by_cap.csv')
summary_period.to_csv('output/summary_by_period.csv')

# CRITICAL ADDITION: PRACTICAL IMPLICATIONS (NOK COST SAVINGS)
print("\n" + "="*80)
print("PRACTICAL IMPLICATIONS: TRADING COST ANALYSIS")
print("="*80)

# Calculate cost savings for realistic trade sizes
trade_sizes = [100000, 500000, 1000000, 5000000]  # NOK
avg_stock_price = 100  # Assume 100 NOK per share

spread_control = summary_period.loc['control_year', 'Spread_Mean']
spread_summer = summary_period.loc['summer_holiday', 'Spread_Mean']
spread_diff = spread_summer - spread_control

print(f"\nSpread difference (Summer vs. Control): {spread_diff:.6f}")
print(f"Average stock price assumed: {avg_stock_price} NOK\n")

print("Cost Impact per Trade (one-way):")
print(f"{'Trade Size (NOK)':>20} {'Control Cost':>15} {'Summer Cost':>15} {'Savings':>15}")
print("-"*70)

for trade_size in trade_sizes:
    shares = trade_size / avg_stock_price
    cost_control = shares * avg_stock_price * spread_control / 2  # Half-spread for one-way
    cost_summer = shares * avg_stock_price * spread_summer / 2
    savings = cost_control - cost_summer
    
    print(f"{trade_size:>20,} {cost_control:>14.2f} NOK {cost_summer:>14.2f} NOK {savings:>14.2f} NOK")

print("\n💡 INTERPRETATION:")
if spread_diff < 0:
    print(f"   Trading during summer holidays saves approximately {abs(spread_diff/spread_control)*100:.1f}%")
    print(f"   on transaction costs due to narrower spreads.")
else:
    print(f"   Trading during summer holidays costs approximately {(spread_diff/spread_control)*100:.1f}% more")
    print(f"   due to wider spreads (consistent with Harris's dealer hypothesis).")


# FIGURE 1: CHRISTMAS & EASTER ANALYSIS
print("\n" + "-"*80)
print("GENERATING FIGURE 1: CHRISTMAS & EASTER")
print("-"*80)

christmas_easter_periods = ['control_year', 'pre_easter', 'post_easter',
                            'pre_christmas', 'christmas', 'post_christmas']
df_ce = df[df['period_label'].isin(christmas_easter_periods)].copy()
df_ce['period_label'] = pd.Categorical(df_ce['period_label'],
                                       categories=christmas_easter_periods,
                                       ordered=True)

# Figure 1A: Spread (added confidence intervals)
plt.figure(figsize=(14, 8))
sns.pointplot(x='period_label', y='Spread', hue='Cap_Group', data=df_ce,
             hue_order=['Large Cap', 'Mid Cap', 'Small Cap'],
             dodge=True,
             palette='viridis',
             errorbar='ci',
             capsize=0.1)
plt.title(f'Figure 1A: Mean Spread (Christmas & Easter Periods) | {SPREAD_METHOD_NAME}', fontsize=18, fontweight='bold')
plt.xlabel('Period', fontsize=14)
plt.ylabel('Mean Spread', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Cap Group', title_fontsize=12, fontsize=11)
plt.tight_layout()
plt.savefig("output/Figure_1A_CE_Spread.png", dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_1A_CE_Spread.png")

# Figure 1B: Volume
volume_ce = df_ce.groupby(['period_label', 'Cap_Group'], observed=False)['Volume'].mean().reset_index()
volume_ce['Volume_millions'] = volume_ce['Volume'] / 1e6
plt.figure(figsize=(14, 8))
sns.barplot(x='period_label', y='Volume_millions', hue='Cap_Group', data=volume_ce,
           hue_order=['Large Cap', 'Mid Cap', 'Small Cap'],
           palette='magma',
           errorbar='ci')
plt.title(f'Figure 1B: Mean Daily Volume (Christmas & Easter Periods) | {SPREAD_METHOD_NAME}', fontsize=18, fontweight='bold')
plt.xlabel('Period', fontsize=14)
plt.ylabel('Mean Volume (millions)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Cap Group', title_fontsize=12, fontsize=11)
plt.tight_layout()
plt.savefig("output/Figure_1B_CE_Volume.png", dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_1B_CE_Volume.png")

# FIGURE 2: SUMMER ANALYSIS
print("\n" + "-"*80)
print("GENERATING FIGURE 2: SUMMER HOLIDAY")
print("-"*80)

summer_periods = ['control_year', 'summer_excl_holiday', 'summer_holiday']
df_summer = df[df['period_label'].isin(summer_periods)].copy()
df_summer['period_label'] = pd.Categorical(df_summer['period_label'],
                                           categories=summer_periods,
                                           ordered=True)

# Figure 2A: Spread
plt.figure(figsize=(10, 7))
sns.pointplot(x='period_label', y='Spread', hue='Cap_Group', data=df_summer,
             hue_order=['Large Cap', 'Mid Cap', 'Small Cap'],
             dodge=True,
             palette='viridis',
             errorbar='ci',
             capsize=0.1)
plt.title(f'Figure 2A: Mean Spread (Summer Holiday) | {SPREAD_METHOD_NAME}', fontsize=18, fontweight='bold')
plt.xlabel('Period', fontsize=14)
plt.ylabel('Mean Spread', fontsize=14)
plt.xticks(rotation=0)
plt.legend(title='Cap Group', title_fontsize=12, fontsize=11)
plt.tight_layout()
plt.savefig("output/Figure_2A_Summer_Spread.png", dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_2A_Summer_Spread.png")

# Figure 2B: Volume
volume_summer = df_summer.groupby(['period_label', 'Cap_Group'], observed=False)['Volume'].mean().reset_index()
volume_summer['Volume_millions'] = volume_summer['Volume'] / 1e6
plt.figure(figsize=(10, 7))
sns.barplot(x='period_label', y='Volume_millions', hue='Cap_Group', data=volume_summer,
           hue_order=['Large Cap', 'Mid Cap', 'Small Cap'],
           palette='magma',
           errorbar='ci')
plt.title(f'Figure 2B: Mean Daily Volume (Summer Holiday) | {SPREAD_METHOD_NAME}', fontsize=18, fontweight='bold')
plt.xlabel('Period', fontsize=14)
plt.ylabel('Mean Volume (millions)', fontsize=14)
plt.xticks(rotation=0)
plt.legend(title='Cap Group', title_fontsize=12, fontsize=11)
plt.tight_layout()
plt.savefig("output/Figure_2B_Summer_Volume.png", dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_2B_Summer_Volume.png")

# FIGURE 3: P-VALUE HEATMAPS WITH MULTIPLE TESTING CORRECTION
print("\n" + "-"*80)
print("GENERATING FIGURE 3: P-VALUE HEATMAPS (WITH MULTIPLE CORRECTIONS)")
print("-"*80)

# Import multiple testing correction
from statsmodels.stats.multitest import multipletests

p_values_spread = {}
p_values_volume = {}

# Collect all raw p-values for correction
all_p_spread = []
all_p_volume = []
test_info = []  # To keep track of which test corresponds to which p-value

for group in ['Large Cap', 'Mid Cap', 'Small Cap']:
    p_values_spread[group] = {}
    p_values_volume[group] = {}
    
    control = df_ce[(df_ce['period_label'] == 'control_year') & (df_ce['Cap_Group'] == group)]
    
    for period in [p for p in christmas_easter_periods if p != 'control_year']:
        event = df_ce[(df_ce['period_label'] == period) & (df_ce['Cap_Group'] == group)]
        
        if len(event) > 1 and len(control) > 1:
            _, p_spread = ttest_ind(event['Spread'].dropna(),
                                   control['Spread'].dropna(),
                                   equal_var=False)
            _, p_volume = ttest_ind(event['Volume'].dropna(),
                                   control['Volume'].dropna(),
                                   equal_var=False)
            
            all_p_spread.append(p_spread)
            all_p_volume.append(p_volume)
            test_info.append((group, period))
            
            p_values_spread[group][period] = p_spread
            p_values_volume[group][period] = p_volume

# Multiple Testing Correction

# Method 1: Bonferroni Correction (Conservative)
print("\n--- Applying Bonferroni Correction ---")
print(f'Total tests performed: {len(all_p_spread)}')
print(f'Bonferroni-corrected alpha level: {0.05 / len(all_p_spread):.5f}\n')

reject_spread_bonf, pvals_corrected_spread_bonf, _, _ = multipletests(
    all_p_spread, alpha=0.05, method='bonferroni'
)
reject_volume_bonf, pvals_corrected_volume_bonf, _, _ = multipletests(
    all_p_volume, alpha=0.05, method='bonferroni'
)

# Method 2: Benjamini-Hochberg (FDR) Correction (Less Conservative)
print("--- Applying Benjamini-Hochberg (FDR) Correction ---")
reject_spread_fdr, pvals_corrected_spread_fdr, _, _ = multipletests(
    all_p_spread, alpha=0.05, method='fdr_bh'
)
reject_volume_fdr, pvals_corrected_volume_fdr, _, _ = multipletests(
    all_p_volume, alpha=0.05, method='fdr_bh'
)
print("FDR correction applied.\n")

# Create DataFrames for Heatmaps and Comparison

# Create DataFrame for raw p-values (for heatmap annotation)
p_values_spread_df = pd.DataFrame(p_values_spread).T
p_values_volume_df = pd.DataFrame(p_values_volume).T

# Create DataFrames for corrected p-values
p_values_corrected_df_spread_bonf = pd.DataFrame(index=p_values_spread_df.index, columns=p_values_spread_df.columns)
p_values_corrected_df_volume_bonf = pd.DataFrame(index=p_values_volume_df.index, columns=p_values_volume_df.columns)
p_values_corrected_df_spread_fdr = pd.DataFrame(index=p_values_spread_df.index, columns=p_values_spread_df.columns)
p_values_corrected_df_volume_fdr = pd.DataFrame(index=p_values_volume_df.index, columns=p_values_volume_df.columns)

# Populate corrected p-value DataFrames
for i, (group, period) in enumerate(test_info):
    p_values_corrected_df_spread_bonf.loc[group, period] = pvals_corrected_spread_bonf[i]
    p_values_corrected_df_volume_bonf.loc[group, period] = pvals_corrected_volume_bonf[i]
    p_values_corrected_df_spread_fdr.loc[group, period] = pvals_corrected_spread_fdr[i]
    p_values_corrected_df_volume_fdr.loc[group, period] = pvals_corrected_volume_fdr[i]

# --- Print Comparison Tables ---
print("\n" + "-"*80)
print("COMPARISON OF CORRECTED P-VALUES (SPREAD)")
print("-"*80)
print("\n--- Bonferroni Corrected P-Values (Spread) ---")
print(p_values_corrected_df_spread_bonf.astype(float).round(4))
print("\n--- FDR Corrected P-Values (Spread) ---")
print(p_values_corrected_df_spread_fdr.astype(float).round(4))

print("\n" + "-"*80)
print("COMPARISON OF CORRECTED P-VALUES (VOLUME)")
print("-"*80)
print("\n--- Bonferroni Corrected P-Values (Volume) ---")
print(p_values_corrected_df_volume_bonf.astype(float).round(4))
print("\n--- FDR Corrected P-Values (Volume) ---")
print(p_values_corrected_df_volume_fdr.astype(float).round(4))

# Figure 3A: Spread p-values with Bonferroni-corrected significance
fig, ax = plt.subplots(figsize=(12, 7))
sns.heatmap(p_values_spread_df, annot=True, cmap='viridis_r', fmt=".3f",
            linewidths=.5, ax=ax, cbar_kws={'label': 'P-value (uncorrected)'})

# Highlight Bonferroni-significant cells (This part remains unchanged)
for i in range(len(p_values_corrected_df_spread_bonf.index)):
    for j in range(len(p_values_corrected_df_spread_bonf.columns)):
        val = p_values_corrected_df_spread_bonf.iloc[i, j]
        if val < 0.05:
            ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False,
                                      edgecolor='red', lw=3))

ax.set_title(f'Figure 3A: P-values for Spread (vs. Control) | {SPREAD_METHOD_NAME}\nRed border = p < 0.05 (Bonferroni-corrected)',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig("output/Figure_3A_PValues_Spread.png", dpi=300, bbox_inches='tight')
plt.close()
print("\nSaved: Figure_3A_PValues_Spread.png")

# Figure 3B: Volume p-values with Bonferroni-corrected significance
fig, ax = plt.subplots(figsize=(12, 7))
sns.heatmap(p_values_volume_df, annot=True, cmap='plasma_r', fmt=".3f",
            linewidths=.5, ax=ax, cbar_kws={'label': 'P-value (uncorrected)'})

# Highlight Bonferroni-significant cells (This part remains unchanged)
for i in range(len(p_values_corrected_df_volume_bonf.index)):
    for j in range(len(p_values_corrected_df_volume_bonf.columns)):
        val = p_values_corrected_df_volume_bonf.iloc[i, j]
        if val < 0.05:
            ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False,
                                      edgecolor='red', lw=3))

ax.set_title(f'Figure 3B: P-values for Volume (vs. Control) | {SPREAD_METHOD_NAME}\nRed border = p < 0.05 (Bonferroni-corrected)',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig("output/Figure_3B_PValues_Volume.png", dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_3B_PValues_Volume.png")

# FIGURE 4: HEATMAP - SPREAD BY CAP AND PERIOD
print("\n" + "-"*80)
print("GENERATING FIGURE 4: HEATMAP")
print("-"*80)

period_order = ['control_year', 'pre_easter', 'post_easter',
                'pre_christmas', 'christmas', 'post_christmas',
                'summer_excl_holiday', 'summer_holiday']
period_order = [p for p in period_order if p in df['period_label'].unique()]

heatmap_data = df.groupby(['Cap_Group', 'period_label'])['Spread'].mean().unstack()
heatmap_data = heatmap_data[period_order]

fig, ax = plt.subplots(figsize=(16, 6))

sns.heatmap(heatmap_data,
            annot=True,
            fmt='.5f',
            cmap='YlOrRd',
            linewidths=0.5,
            cbar_kws={'label': 'Avg Spread'},
            ax=ax,
            annot_kws={'fontsize': 11})

ax.set_title(f'Figure 4: Average Spread by Cap Group and Period | {SPREAD_METHOD_NAME}',
             fontweight='bold',
             fontsize=18,
             pad=15)

ax.set_xlabel('Period', fontsize=14, fontweight='bold')
ax.set_ylabel('Cap Group', fontsize=14, fontweight='bold')

ax.tick_params(axis='y', labelsize=13)
ax.tick_params(axis='x', labelsize=12)

ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

plt.tight_layout()
plt.savefig('output/Figure_4_Heatmap_Spread.png', dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_4_Heatmap_Spread.png")

# FIGURE 5: TIME SERIES - MONTHLY AVERAGE SPREAD
print("\n" + "-"*80)
print(f"GENERATING FIGURE 5: TIME SERIES | {SPREAD_METHOD_NAME}")
print("-"*80)

df['YearMonth'] = df['Date'].dt.to_period('M')
monthly_spread = df.groupby('YearMonth')['Spread'].mean()

fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(monthly_spread.index.to_timestamp(), monthly_spread.values,
        color='darkblue', linewidth=1.5, alpha=0.8)
ax.set_title(f'Figure 5: Monthly Average Spread (2014-2024) | {SPREAD_METHOD_NAME}', fontweight='bold', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Average Spread', fontsize=14)
ax.grid(alpha=0.3)

covid_start = pd.Timestamp('2020-03-01')
covid_end = pd.Timestamp('2020-12-31')
ax.axvspan(covid_start, covid_end, alpha=0.2, color='red', label='COVID-19')
ax.legend(fontsize=12)

plt.tight_layout()
plt.savefig('output/Figure_5_Timeseries_Monthly.png', dpi=300, bbox_inches='tight')
plt.close()
print("Saved: Figure_5_Timeseries_Monthly.png")

print("\n" + "-"*80)
print("COMPLETE - 5 essential figures created")
print("-"*80)
print("\nAll outputs saved to 'output/' folder")


--------------------------------------------------------------------------------
DESCRIPTIVE STATISTICS AND VISUALIZATION
--------------------------------------------------------------------------------

Total observations: 119,138
Date range: 2014-01-03 00:00:00 to 2024-12-30 00:00:00
Unique tickers: 90
Trading days: 2,757

EFFECT SIZE ANALYSIS (COHEN'S D)

Cohen's d Effect Sizes (vs. Control Period):
  Summer Holiday:    d = -0.1261
  Christmas:         d = 0.0268
  Pre-Easter:        d = -0.0011
  Post-Easter:       d = -0.0301

Interpretation: |d| > 0.5 = medium to large effect

HARRIS FRAMEWORK: DEALER BEHAVIOR HYPOTHESIS

Testing Harris's prediction: Spreads should increase when volume decreases
(Lower dealer participation and higher inventory risk)

Spread by Volume Quartile:
                 Spread_Mean  Spread_Median  N_Obs    Avg_Volume
Volume_Quartile                                                 
Q1_Lowest           0.017639       0.011911  29785  7.352360e+03
Q2_Low     

In [16]:
# SCRIPT 3: OLS REGRESSION ANALYSIS WITH DYNAMIC DESCRIPTIVE TITLES

print("-" * 80)
print("SCRIPT 3: OLS REGRESSION ANALYSIS")
print(f"Using Dependent Variable: {SPREAD_METHOD_NAME}")
print("-" * 80)

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

# Load data
try:
    df = pd.read_csv(INPUT_CSV, parse_dates=['Date'])
    print(f"Successfully loaded data from {INPUT_CSV}")
except FileNotFoundError:
    print(f"FATAL ERROR: The input file '{INPUT_CSV}' was not found. Please ensure SCRIPT 1 has been run successfully.")
    exit()

# Data Preparation
print("\n--- Preparing data for regression analysis ---")
df = df.sort_values(['Ticker', 'Date'])
df['Return'] = df.groupby('Ticker')['Close'].pct_change()
df['Volatility'] = df.groupby('Ticker')['Return'].transform(lambda x: x.shift(1).rolling(window=20, min_periods=10).std())
df['Log_Volume'] = np.log1p(df['Volume'])
df['Log_Volume_lag1'] = df.groupby('Ticker')['Log_Volume'].shift(1)

# Holiday and Market Cap Dummies
df['Pre_Easter'] = (df['period_label'] == 'pre_easter').astype(int)
df['Post_Easter'] = (df['period_label'] == 'post_easter').astype(int)
df['Christmas'] = (df['period_label'] == 'christmas').astype(int)
df['Pre_Christmas'] = (df['period_label'] == 'pre_christmas').astype(int)
df['Post_Christmas'] = (df['period_label'] == 'post_christmas').astype(int)
df['Summer'] = (df['period_label'] == 'summer_holiday').astype(int)
df['Large_Cap'] = (df['Cap_Group'] == 'Large Cap').astype(int)
df['Mid_Cap'] = (df['Cap_Group'] == 'Mid Cap').astype(int)

# Create regression sample
df_reg = df.dropna(subset=['Spread', 'Log_Volume_lag1', 'Volatility']).copy()
print(f"Regression sample created with {len(df_reg):,} observations.")

# VIF MULTICOLLINEARITY CHECK
print("\n--- VIF MULTICOLLINEARITY DIAGNOSTICS ---")
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_vars = ['Pre_Easter', 'Post_Easter', 'Christmas', 'Pre_Christmas', 'Post_Christmas', 
            'Summer', 'Log_Volume_lag1', 'Volatility', 'Large_Cap', 'Mid_Cap']
X_vif = df_reg[vif_vars].copy()

vif_data = pd.DataFrame()
vif_data["Variable"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)

print("Variance Inflation Factors:")
print(vif_data.to_string(index=False))

high_vif = vif_data[vif_data['VIF'] > 10]
if len(high_vif) > 0:
    print("\nWARNING: High multicollinearity detected (VIF > 10)")
    print(high_vif.to_string(index=False))

# Models 1A-C: Baseline Holiday Effects by Market Cap
print("\n--- MODELS 1A-1C: BASELINE HOLIDAY EFFECTS BY MARKET CAP ---")
baseline_models = {}
for i, cap in enumerate(['Large Cap', 'Mid Cap', 'Small Cap']):
    print(f"\nProcessing Model 1{chr(65+i)}: {cap}")
    df_cap = df_reg[df_reg['Cap_Group'] == cap].copy()
    
    y = df_cap['Spread']
    X = df_cap[['Pre_Easter', 'Post_Easter', 'Christmas', 'Pre_Christmas', 'Post_Christmas', 'Summer']]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df_cap['Ticker']})
    baseline_models[cap] = model
    
    # Create and save summary as an image with the dynamic title
    fig, ax = plt.subplots(figsize=(12, 10))
    ax.axis('off')
    title_text = f"Table 1{chr(65+i)}: Baseline Spread Regression - {cap} Stocks\nDependent Variable: {SPREAD_METHOD_NAME}"
    ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
    summary_text = str(model.summary())
    ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 9}, va='top', ha='left', transform=ax.transAxes)
    
    filename = f"output/Table_1{chr(65+i)}_Baseline_{cap.replace(' ', '_')}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"SUCCESS: Saved model summary to {filename}")

# Model 2: Full Sample with Controls
print("\n--- MODEL 2: SPREAD WITH CONTROL VARIABLES ---")
X2 = df_reg[['Pre_Easter', 'Post_Easter', 'Christmas', 'Pre_Christmas', 'Post_Christmas', 'Summer',
             'Log_Volume_lag1', 'Volatility', 'Large_Cap', 'Mid_Cap']]
X2 = sm.add_constant(X2)
y2 = df_reg['Spread']
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': df_reg['Ticker']})

fig, ax = plt.subplots(figsize=(12, 11))
ax.axis('off')
title_text = (f'Table 2: Spread Determinants with Control Variables (Full Sample)\n'
              f'Dependent Variable: {SPREAD_METHOD_NAME} | Controls: Lagged Volume, Volatility, Market Cap')
ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
summary_text = str(model2.summary())
ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 9}, va='top', ha='left', transform=ax.transAxes)
filename_2 = 'output/Table_2_Spread_Full_Controls.png'
plt.savefig(filename_2, dpi=300, bbox_inches='tight')
plt.close()
print(f"SUCCESS: Saved model summary to {filename_2}")

# MODEL 3: STOCK FIXED EFFECTS
print("\n--- MODEL 3: OLS WITH STOCK FIXED EFFECTS ---")
stock_dummies = pd.get_dummies(df_reg['Ticker'], prefix='Stock', drop_first=True, dtype=float)

X3 = pd.concat([
    df_reg[['Pre_Easter', 'Post_Easter', 'Christmas', 'Pre_Christmas', 'Post_Christmas', 'Summer',
            'Log_Volume_lag1', 'Volatility']].astype(float),
    stock_dummies
], axis=1)
X3 = sm.add_constant(X3)

model3 = sm.OLS(df_reg['Spread'].astype(float), X3).fit(cov_type='cluster', cov_kwds={'groups': df_reg['Ticker']})

print(f"Model 3: R-squared = {model3.rsquared:.4f}, N = {model3.nobs:.0f}")

coef_interest = ['const', 'Pre_Easter', 'Post_Easter', 'Christmas', 'Pre_Christmas', 
                 'Post_Christmas', 'Summer', 'Log_Volume_lag1', 'Volatility']
coef_table = pd.DataFrame({
    'Coefficient': model3.params[coef_interest],
    'Std Error': model3.bse[coef_interest],
    'P>|t|': model3.pvalues[coef_interest]
}).round(6)

print("\nKey Coefficients:")
print(coef_table.to_string())

fig, ax = plt.subplots(figsize=(12, 10))
ax.axis('off')
title_text = f'Table 3: Spread Regression with Stock Fixed Effects\nDependent Variable: {SPREAD_METHOD_NAME}'
ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
summary_text = f"""OLS Regression Results (with Stock FE)
R-squared: {model3.rsquared:.4f}
Observations: {model3.nobs:.0f}
Stock FE: {len(stock_dummies.columns)} dummies included

{coef_table.to_string()}

Notes: Standard errors clustered by Ticker
"""
ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 9}, va='top', ha='left', transform=ax.transAxes)
filename_3 = 'output/Table_3_Stock_Fixed_Effects.png'
plt.savefig(filename_3, dpi=300, bbox_inches='tight')
plt.close()
print(f"SUCCESS: Saved model summary to {filename_3}")

# MODEL 4: PANEL REGRESSION
print("\n--- MODEL 4: PANEL REGRESSION WITH ENTITY & TIME FIXED EFFECTS ---")
from linearmodels.panel import PanelOLS

df_panel = df_reg.set_index(['Ticker', 'Date']).copy()
y_panel = df_panel['Spread']
X_panel = df_panel[['Pre_Easter', 'Post_Easter', 'Christmas', 'Pre_Christmas', 
                    'Post_Christmas', 'Summer', 'Log_Volume_lag1', 'Volatility']]

model4 = PanelOLS(y_panel, X_panel, entity_effects=True, time_effects=True, drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True)

print(f"Model 4: R-squared (within) = {model4.rsquared:.4f}, N = {model4.nobs:.0f}")
print("\nPanel Regression Coefficients:")
print(model4.summary.tables[1])

fig, ax = plt.subplots(figsize=(12, 11))
ax.axis('off')
title_text = f'Table 4: Panel Regression with Entity & Time Fixed Effects\nDependent Variable: {SPREAD_METHOD_NAME}'
ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
summary_text = str(model4.summary)
ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 8}, va='top', ha='left', transform=ax.transAxes)
filename_4 = 'output/Table_4_Panel_Entity_Time_FE.png'
plt.savefig(filename_4, dpi=300, bbox_inches='tight')
plt.close()
print(f"SUCCESS: Saved model summary to {filename_4}")

# MODEL COMPARISON
print("\n--- MODEL COMPARISON ---")
comparison_df = pd.DataFrame({
    'Model': ['Model 2 (OLS)', 'Model 3 (Stock FE)', 'Model 4 (Panel FE)'],
    'Summer Coef': [model2.params.get('Summer', np.nan), model3.params.get('Summer', np.nan), model4.params.get('Summer', np.nan)],
    'Summer P-val': [model2.pvalues.get('Summer', np.nan), model3.pvalues.get('Summer', np.nan), model4.pvalues.get('Summer', np.nan)],
    'R-squared': [model2.rsquared, model3.rsquared, model4.rsquared]
})
print(comparison_df.to_string(index=False))
comparison_df.to_csv('output/Model_Comparison_Summary.csv', index=False)

print("\n" + "-"*80)
print("Script finished. All regression tables have been saved as PNG images in the 'output' folder.")
print("-" * 80)


--------------------------------------------------------------------------------
SCRIPT 3: OLS REGRESSION ANALYSIS
Using Dependent Variable: Corwin-Schultz (2012) Spread
--------------------------------------------------------------------------------
Successfully loaded data from oslo_bors_labelled_data.csv

--- Preparing data for regression analysis ---
Regression sample created with 118,148 observations.

--- VIF MULTICOLLINEARITY DIAGNOSTICS ---
Variance Inflation Factors:
       Variable      VIF
Log_Volume_lag1 4.872278
      Large_Cap 2.550172
        Mid_Cap 2.003723
     Volatility 1.851746
         Summer 1.066546
      Christmas 1.046437
  Pre_Christmas 1.024273
     Pre_Easter 1.023075
    Post_Easter 1.021600
 Post_Christmas 1.020296

--- MODELS 1A-1C: BASELINE HOLIDAY EFFECTS BY MARKET CAP ---

Processing Model 1A: Large Cap
SUCCESS: Saved model summary to output/Table_1A_Baseline_Large_Cap.png

Processing Model 1B: Mid Cap
SUCCESS: Saved model summary to output/Table_1B_B

/var/folders/4f/whd5k2z14l1f9v052bj7zhfm0000gn/T/ipykernel_16338/2679274653.py:165: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pre_Easter, Post_Easter, Christmas, Pre_Christmas, Post_Christmas, Summer

  model4 = PanelOLS(y_panel, X_panel, entity_effects=True, time_effects=True, drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True)


Model 4: R-squared (within) = 0.0442, N = 118148

Panel Regression Coefficients:
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
Log_Volume_lag1     0.0014     0.0003     5.6701     0.0000      0.0009      0.0020
Volatility          0.0682     0.0181     3.7601     0.0002      0.0327      0.1038
SUCCESS: Saved model summary to output/Table_4_Panel_Entity_Time_FE.png

--- MODEL COMPARISON ---
             Model  Summer Coef  Summer P-val  R-squared
     Model 2 (OLS)    -0.001510  4.877897e-07   0.138996
Model 3 (Stock FE)    -0.001333  1.314628e-06   0.254227
Model 4 (Panel FE)          NaN           NaN   0.044217

--------------------------------------------------------------------------------
Script finished. All regression tables have been saved as PNG images in the 'output' folder

In [17]:
# SCRIPT 4: ROBUSTNESS CHECK - EXCLUDE COVID PERIOD
print("-" * 80)
print("SCRIPT 4: ROBUSTNESS CHECK - EXCLUDING COVID PERIOD")
print("-" * 80)

# Define the input file and output directory
INPUT_CSV = "oslo_bors_labelled_data.csv"
output_dir = 'output'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

# Load the data
try:
    df = pd.read_csv(INPUT_CSV, parse_dates=['Date'])
    print(f"Successfully loaded data from '{INPUT_CSV}'")
except FileNotFoundError:
    print(f"FATAL ERROR: The input file '{INPUT_CSV}' was not found. Please ensure it's in the correct directory.")
    exit()

# Exclude the COVID-19 Period
print("\n--- Excluding the entire year 2020 for robustness check ---")
df_robust = df[df['Date'].dt.year != 2020].copy()
print(f"Original sample size: {len(df):,} observations.")
print(f"Robust sample size (2020 excluded): {len(df_robust):,} observations.")
print(f"Removed {len(df) - len(df_robust):,} observations.")

# Re-create all necessary variables
print("\n--- Re-creating analysis variables for the robust sample ---")
df_robust = df_robust.sort_values(['Ticker', 'Date'])
df_robust['Return'] = df_robust.groupby('Ticker')['Close'].pct_change()
df_robust['Volatility'] = df_robust.groupby('Ticker')['Return'].transform(lambda x: x.shift(1).rolling(window=20, min_periods=10).std())
df_robust['Log_Volume'] = np.log1p(df_robust['Volume'])
df_robust['Log_Volume_lag1'] = df_robust.groupby('Ticker')['Log_Volume'].shift(1)
df_robust['Large_Cap'] = (df_robust['Cap_Group'] == 'Large Cap').astype(int)
df_robust['Mid_Cap'] = (df_robust['Cap_Group'] == 'Mid Cap').astype(int)
df_robust['Summer'] = (df_robust['period_label'] == 'summer_holiday').astype(int)
df_robust['Christmas'] = (df_robust['period_label'] == 'christmas').astype(int)

# Drop rows with missing values
initial_len = len(df_robust)
df_robust = df_robust.dropna(subset=['Spread', 'Log_Volume_lag1', 'Volatility']).copy()
print(f"Removed {initial_len - len(df_robust):,} rows with missing values needed for regression.")
print(f"Final robust sample size: {len(df_robust):,} observations.")

# VIF MULTICOLLINEARITY CHECK
print("\n--- VIF MULTICOLLINEARITY DIAGNOSTICS (ROBUST SAMPLE) ---")
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_vars = ['Summer', 'Christmas', 'Log_Volume_lag1', 'Volatility', 'Large_Cap', 'Mid_Cap']
X_vif = df_robust[vif_vars].copy()

vif_data = pd.DataFrame()
vif_data["Variable"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)

print("Variance Inflation Factors:")
print(vif_data.to_string(index=False))

high_vif = vif_data[vif_data['VIF'] > 10]
if len(high_vif) > 0:
    print("\nWARNING: High multicollinearity detected (VIF > 10)")
    print(high_vif.to_string(index=False))

# MODEL 1: BASELINE OLS
print("\n--- MODEL 1 (ROBUST): BASELINE OLS ---")
y = df_robust['Spread']
X = df_robust[['Summer', 'Christmas', 'Log_Volume_lag1', 'Volatility', 'Large_Cap', 'Mid_Cap']]
X = sm.add_constant(X)

model_robust = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df_robust['Ticker']})

print(f"Model 1 (Robust): R-squared = {model_robust.rsquared:.4f}, N = {model_robust.nobs:.0f}")

# Save Model 1
fig, ax = plt.subplots(figsize=(12, 11))
ax.axis('off')
title_text = ('Table 5A: Robustness Check - Excluding COVID-19 Period (2020)\n'
              f'Dependent Variable: {SPREAD_METHOD_NAME} | Controls: Lagged Volume, Volatility, Market Cap')
ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
summary_text = str(model_robust.summary())
ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 9}, va='top', ha='left', transform=ax.transAxes)
output_filename_png = 'output/Table_5A_Robustness_No_COVID.png'
plt.savefig(output_filename_png, dpi=300, bbox_inches='tight')
plt.close()
print(f"SUCCESS: Saved to '{output_filename_png}'")

# MODEL 2: STOCK FIXED EFFECTS
print("\n--- MODEL 2 (ROBUST): OLS WITH STOCK FIXED EFFECTS ---")
stock_dummies = pd.get_dummies(df_robust['Ticker'], prefix='Stock', drop_first=True, dtype=float)

X2 = pd.concat([
    df_robust[['Summer', 'Christmas', 'Log_Volume_lag1', 'Volatility']].astype(float),
    stock_dummies
], axis=1)
X2 = sm.add_constant(X2)

model_robust_fe = sm.OLS(df_robust['Spread'].astype(float), X2).fit(cov_type='cluster', cov_kwds={'groups': df_robust['Ticker']})

print(f"Model 2 (Robust FE): R-squared = {model_robust_fe.rsquared:.4f}, N = {model_robust_fe.nobs:.0f}")

coef_interest = ['const', 'Summer', 'Christmas', 'Log_Volume_lag1', 'Volatility']
coef_table = pd.DataFrame({
    'Coefficient': model_robust_fe.params[coef_interest],
    'Std Error': model_robust_fe.bse[coef_interest],
    'P>|t|': model_robust_fe.pvalues[coef_interest]
}).round(6)

print("\nKey Coefficients:")
print(coef_table.to_string())

# Save Model 2
fig, ax = plt.subplots(figsize=(12, 10))
ax.axis('off')
title_text = f'Table 5B: Robustness with Stock Fixed Effects (No COVID)\nDependent Variable: {SPREAD_METHOD_NAME}'
ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
summary_text = f"""OLS Regression Results (with Stock FE, No COVID)
R-squared: {model_robust_fe.rsquared:.4f}
Observations: {model_robust_fe.nobs:.0f}
Stock FE: {len(stock_dummies.columns)} dummies included

{coef_table.to_string()}

Notes: Standard errors clustered by Ticker
"""
ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 9}, va='top', ha='left', transform=ax.transAxes)
filename_2 = 'output/Table_5B_Robustness_Stock_FE.png'
plt.savefig(filename_2, dpi=300, bbox_inches='tight')
plt.close()
print(f"SUCCESS: Saved to '{filename_2}'")

# MODEL 3: PANEL REGRESSION
print("\n--- MODEL 3 (ROBUST): PANEL REGRESSION WITH ENTITY & TIME FIXED EFFECTS ---")
from linearmodels.panel import PanelOLS

df_panel = df_robust.set_index(['Ticker', 'Date']).copy()
y_panel = df_panel['Spread']
X_panel = df_panel[['Summer', 'Christmas', 'Log_Volume_lag1', 'Volatility']]

model_robust_panel = PanelOLS(y_panel, X_panel, entity_effects=True, time_effects=True, drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True)

print(f"Model 3 (Robust Panel): R-squared (within) = {model_robust_panel.rsquared:.4f}, N = {model_robust_panel.nobs:.0f}")
print("\nPanel Regression Coefficients:")
print(model_robust_panel.summary.tables[1])

# Save Model 3
fig, ax = plt.subplots(figsize=(12, 11))
ax.axis('off')
title_text = f'Table 5C: Robustness Panel Regression (No COVID)\nDependent Variable: {SPREAD_METHOD_NAME}'
ax.text(0.5, 0.98, title_text, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transAxes)
summary_text = str(model_robust_panel.summary)
ax.text(0.01, 0.93, summary_text, fontdict={'fontname': 'monospace', 'fontsize': 8}, va='top', ha='left', transform=ax.transAxes)
filename_3 = 'output/Table_5C_Robustness_Panel_FE.png'
plt.savefig(filename_3, dpi=300, bbox_inches='tight')
plt.close()
print(f"SUCCESS: Saved to '{filename_3}'")

# MODEL COMPARISON
print("\n--- ROBUSTNESS MODEL COMPARISON ---")
comparison_df = pd.DataFrame({
    'Model': ['Robust OLS', 'Robust Stock FE', 'Robust Panel FE'],
    'Summer Coef': [model_robust.params.get('Summer', np.nan), model_robust_fe.params.get('Summer', np.nan), model_robust_panel.params.get('Summer', np.nan)],
    'Summer P-val': [model_robust.pvalues.get('Summer', np.nan), model_robust_fe.pvalues.get('Summer', np.nan), model_robust_panel.pvalues.get('Summer', np.nan)],
    'Christmas Coef': [model_robust.params.get('Christmas', np.nan), model_robust_fe.params.get('Christmas', np.nan), model_robust_panel.params.get('Christmas', np.nan)],
    'Christmas P-val': [model_robust.pvalues.get('Christmas', np.nan), model_robust_fe.pvalues.get('Christmas', np.nan), model_robust_panel.pvalues.get('Christmas', np.nan)],
    'R-squared': [model_robust.rsquared, model_robust_fe.rsquared, model_robust_panel.rsquared]
})
print("\nRobustness Check - Holiday Effects Across Specifications:")
print(comparison_df.to_string(index=False))
comparison_df.to_csv('output/Robustness_Model_Comparison.csv', index=False)
print("\nSUCCESS: Saved comparison to 'output/Robustness_Model_Comparison.csv'")

# Coefficient Summary
print("\n--- Creating coefficient summary table ---")
coef_summary = pd.DataFrame({
    'Variable': ['Summer', 'Christmas', 'Large_Cap', 'Mid_Cap'],
    'Coefficient': model_robust.params[['Summer', 'Christmas', 'Large_Cap', 'Mid_Cap']],
    'Std_Error': model_robust.bse[['Summer', 'Christmas', 'Large_Cap', 'Mid_Cap']],
    'P_value': model_robust.pvalues[['Summer', 'Christmas', 'Large_Cap', 'Mid_Cap']]
}).reset_index(drop=True)

coef_summary['Significance'] = coef_summary['P_value'].apply(lambda x: '***' if x < 0.01 else ('**' if x < 0.05 else ('*' if x < 0.1 else '')))
print("\nKey Coefficients (Robust Sample - Model 1):")
print(coef_summary.to_string(index=False))

output_filename_csv = 'output/Robustness_Coefficients_Summary.csv'
coef_summary.to_csv(output_filename_csv, index=False)
print(f"\nSUCCESS: Coefficient summary saved to '{output_filename_csv}'")

print("\n" + "-"*80)
print("ROBUSTNESS CHECK SCRIPT COMPLETE")
print("-" * 80)


--------------------------------------------------------------------------------
SCRIPT 4: ROBUSTNESS CHECK - EXCLUDING COVID PERIOD
--------------------------------------------------------------------------------
Successfully loaded data from 'oslo_bors_labelled_data.csv'

--- Excluding the entire year 2020 for robustness check ---
Original sample size: 119,138 observations.
Robust sample size (2020 excluded): 107,767 observations.
Removed 11,371 observations.

--- Re-creating analysis variables for the robust sample ---
Removed 990 rows with missing values needed for regression.
Final robust sample size: 106,777 observations.

--- VIF MULTICOLLINEARITY DIAGNOSTICS (ROBUST SAMPLE) ---
Variance Inflation Factors:
       Variable      VIF
Log_Volume_lag1 4.093354
      Large_Cap 2.521512
        Mid_Cap 1.992370
     Volatility 1.339228
         Summer 1.061121
      Christmas 1.041688

--- MODEL 1 (ROBUST): BASELINE OLS ---
Model 1 (Robust): R-squared = 0.0839, N = 106777
SUCCESS: Save

/var/folders/4f/whd5k2z14l1f9v052bj7zhfm0000gn/T/ipykernel_16338/2802494956.py:143: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Summer, Christmas

  model_robust_panel = PanelOLS(y_panel, X_panel, entity_effects=True, time_effects=True, drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True)


Model 3 (Robust Panel): R-squared (within) = 0.0284, N = 106777

Panel Regression Coefficients:
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
Log_Volume_lag1     0.0015     0.0003     6.0546     0.0000      0.0010      0.0020
Volatility          0.0232     0.0126     1.8353     0.0665     -0.0016      0.0479
SUCCESS: Saved to 'output/Table_5C_Robustness_Panel_FE.png'

--- ROBUSTNESS MODEL COMPARISON ---

Robustness Check - Holiday Effects Across Specifications:
          Model  Summer Coef  Summer P-val  Christmas Coef  Christmas P-val  R-squared
     Robust OLS    -0.001388      0.000004        0.001015         0.002892   0.083928
Robust Stock FE    -0.001208      0.000019        0.000764         0.017501   0.251285
Robust Panel FE          NaN           NaN             NaN         